# Cvičení 3

Tématem tohoto cvičení jsou přímé metody řešení soustav lineárních rovnic. Budeme se zabývat dopřednou a zpětnou substitucí a LU rozkladem.

## Dopředná a zpětná substituce

Základní princip přímých metod řešení soustav lineárních rovnic je převod matice soustavy na součin trojúhelníkových matic, např. 
$$\mathsf{A} = \mathsf{L}\mathsf{U}.$$
Soustavu $\mathsf{A}\mathbf{x} = \mathbf{b}$ tak převedeme na tvar
$$\mathsf{L}\mathsf{U}\mathbf{x} = \mathbf{b}$$
a vyřešíme ji ve dvou krocích 
1) $\mathsf{L}\mathbf{y}=\mathbf{b}$
2) $\mathsf{U}\mathbf{x}=\mathbf{y}$
pomocí dopředné a zpětné substituce.

V této části cvičení tedy nejdříve implementujeme funkce pro dopřednou a zpětnou substituci, které později využijeme k řešení soustav.

### Testovací matice

Nejprve vygenerujte testovací matice, pomocí kterých později zjistíte, zda vaše funkce pracují správně. Budete k tomu potřebovat
- funkci z knihovny NumPy, pomocí které vygenerujete náhodnou matici o daném rozměru (znáte z prvního cvičení),
- funkce `triu`, `tril` z knihovny NumPy (zjistěte v dokumentaci nebo pomocí Googlu, k čemu slouží a jak se používají)

In [1]:
# ÚKOL: importujte potřebnou knihovnu NumPy pod zkratkou np
import numpy as np

In [2]:
# ÚKOL: vygenerujte matici A náhodných čísel o rozměrech 4x4
import numpy as np
A = np.random.rand(4, 4)

np.set_printoptions(precision=3) # pro lepší přehlednost
print(A)

[[0.958 0.453 0.89  0.109]
 [0.13  0.518 0.072 0.989]
 [0.775 0.92  0.076 0.501]
 [0.218 0.83  0.11  0.384]]


In [4]:
# ÚKOL: pomocí vhodných funkcí z knihovny NumPy extrahujte z matice A její horní a dolní trojúhelníkovou část.
import numpy as np
A = np.random.rand(4, 4)
U = np.triu(A)
print(U)
print("---------------------")

L = np.tril(A)
print(L)

# Ověřte funkčnost porovnáním s příslušnou částí matice A

[[0.46188922 0.06251362 0.28800523 0.46838382]
 [0.         0.66780806 0.75705364 0.55610883]
 [0.         0.         0.91862866 0.4323584 ]
 [0.         0.         0.         0.3628432 ]]
---------------------
[[0.46188922 0.         0.         0.        ]
 [0.74307947 0.66780806 0.         0.        ]
 [0.80329913 0.25522678 0.91862866 0.        ]
 [0.87327359 0.3489865  0.79675207 0.3628432 ]]


In [ ]:
# ÚKOL: Vytvořte náhodný vektor b délky 4
import numpy as np
b = np.random.rand(4)

print(b)

### Dopředná substituce

Soustavu s dolní trojúhelníkovou maticí $\mathsf{L}\mathbf{x}=\mathbf{b}$ můžeme efektivně řešit pomocí algoritmu dopředné substituce. 

1) Určíme první prvek vektoru řešení
$$x_1 = b_1 / L_{1, 1}$$

2) Zbývající prvky určíme jako 
$$\forall i \in \{2, 3, \ldots, m\}: x_i = (b_i - \sum_{j=1}^{i-1}L_{i, j} x_j) / L_{i, i}$$

Na základě těchto předpisů doplňte chybějící části následujícího kódu. Nezapomeňte ale, že v Pythonu indexujeme od nuly.

In [15]:
# ÚKOL: Doplňte následující kód
import numpy as np

def fsubst(L, b):
    m, n = L.shape             # pomocí vhodné funkce numpy uložte do proměnných m, n počet řádků a sloupců L
    x = np.zeros(n)                # vytvořte vektor samých nul délky n
    
    x[0] = b[0] / L[0, 0]   # první prvek řešení

    for i in range(1, m):

        # nejdříve pomocí vnořeného for cyklu určíme hodnotu sumy z kroku 2:
        suma = 0
        for j in range(0, i):                          # doplňte vhodně meze smyčky
            suma += (L[i, j] * x[j])                     # doplňte
        
        # odečteme sumu od prvku pravé strany a vydělíme diagonálním prvkem
        x[i] = (b[i] - suma)/L[i, i]                        # doplňte

    return x

A = np.random.rand(4, 4)
U = np.triu(A)
L = np.tril(A)
b = np.random.rand(4)
x = fsubst(L, b)
print(x)

[ 0.08626557  1.07020057 -1.00446215  0.87751975]


In [16]:
# Otestujte pomocí L, b
x = fsubst(L, b)
print(x)

[ 0.08626557  1.07020057 -1.00446215  0.87751975]


In [17]:
# ÚKOL: určete řešení pomocí funkce solve z numpy a porovnejte s vašim řešením
x_numpy = np.linalg.solve(L, b)
print(x_numpy)

[ 0.08626557  1.07020057 -1.00446215  0.87751975]


### Zpětná substituce

K vyřešení soustavy s horní trojúhelníkovou maticí $\mathsf{U}\mathbf{x} = \mathbf{b}$ potřebujeme implementovat také algoritmus zpětné substituce. Ten je podobný dopředné substituci, postupuje však od posledního řádku:

1) Určíme poslední prvek vektoru řešení 
$$x_m = b_m / U_{m,m}$$
2) Určíme zbývající prvky
$$\forall i \in \{m-1, m-2, \ldots, 1\}: x_i = (b_i - \sum_{j=i+1}^{m}U_{i, j}x_j) / U_{i, i}$$

Implementujte v následující buňce vaši verzi zpětné substituce.

**Poznámka** Při implementaci se vám bude hodit vědět, že funkce range umožňuje iterovat i "pozpátku". Vyzkoušejte si v následující buňce vytisknout přes jaká *i* iterují tyto `for` cykly:
- `for i in range(6, 0, -1)`
- `for i in range(6, -1, -1)`
- `for i in range(6, 0, -2)`
- `for i in range(0, -6, -1)`

In [18]:
# ÚKOL: Vytiskněte proměnnou i z příkladů v předchozí buňce.
for i in range(6, 0, -1):
    print(i)
for i in range(6, -1, -1):
    print(i)
for i in range(6, 0, -2):
    print(i)
for i in range(0, -6, -1):
    print(i)


6
5
4
3
2
1
6
5
4
3
2
1
0
6
4
2
0
-1
-2
-3
-4
-5


In [26]:
# ÚKOL: Implementujte algoritmus zpětné substituce.
import numpy as np

def b_subst(U, b):
    m, n = U.shape            
    x = np.zeros(m)              
    
    x[n-1] = b[n-1] / U[n-1, n-1]

    for i in range(n-2, -1, -1):
        suma = 0
        for j in range(i+1, n):
            suma += U[i,j] * x[j]
        
        x[i] = (b[i] - suma)/U[i,i]
    
    return x
    

A = np.random.rand(4, 4)
U = np.triu(A)
L = np.tril(A)
b = np.random.rand(4)

In [27]:
# ÚKOL: ověřte pomocí numpy, že vaše funkce vrací správný výsledek

x2 = b_subst(U, b)
x2_numpy = np.linalg.solve(U, b)
print(x2)
print(x2_numpy)

[ 3.86926863 -3.09153792 -1.28155527  4.13049394]
[ 3.86926863 -3.09153792 -1.28155527  4.13049394]


# Domácí úkol č. 1

Algoritmus LU rozkladu lze popsat např. následujícím pseudokódem.

```
U = A, L = I
for k = 1 to m-1 do
    for j = k+1 to m do
        L(j, k) = U(j, k)/U(k, k)
        U(j, k:m) = U(j, k:m) - L(j, k)U(k, k:m)
    end for
end for
```

**Pozor**: značení `k:m` zde znamená sloupce od `k` po `m` *včetně*.

Vašim úkolem je:
1) Na základě uvedeného pseudokódu implementovat LU rozklad bez pivotizace do metody `L, U = my_lu(A)`. Metoda tedy bude mít na vstupu čtvercovou matici A a na výstupu dolní trojúhelníkovou matici L a horní trojúhelníkovou matici U takové, že `A = LU`.
2) Sestavit náhodnou matici A o rozměrech 5x5 a pomocí ní ověřit, že vaše metoda funguje správně (`A - L @ U` by mělo vrátit matici s nulovými či velmi malými prvky).
3) Sestavit náhodný vektor pravé strany `b` o délce 5. 
4) Pomocí vaší metody `my_lu` a na cvičení implementovaných metod `f_subst` a `b_subst` vyřešit soustavu `Ax = b`.
5) Ověřit pomocí metody `np.linalg.solve`, že jste nalezli správné řešení. 

**Odevzdání**: Celý tento notebook s řešením zašlete na e-mail vyučujícího do 23. 3. 16:00. Jako předmět e-mailu uveďte "NLA1 DU1".

In [1]:
# Doplňte podle bodu 1)
import numpy as np

def fsubst(L, b):
    m, n = L.shape       
    x = np.zeros(n)     
    x[0] = b[0] / L[0, 0]

    for i in range(1, m):
        suma = 0
        for j in range(0, i):                        
            suma += (L[i, j] * x[j])                    
        x[i] = (b[i] - suma)/L[i, i]             

    return x

def b_subst(U, b):
    m, n = U.shape            
    x = np.zeros(m)              
    
    x[n-1] = b[n-1] / U[n-1, n-1]

    for i in range(n-2, -1, -1):
        suma = 0
        for j in range(i+1, n):
            suma += U[i,j] * x[j]
        
        x[i] = (b[i] - suma)/U[i,i]
    
    return x

def my_lu(A):
    U = np.copy(A)
    m, n = A.shape
    L = np.eye(n)
    for k in range(0, n-1):
        for j in range(k+1, n):
            L[j,k] = U[j,k]/U[k,k]
            for i in range(k, n):
                U[j,i] = U[j,i] - (L[j,k]*U[k,i])
    list = (L, U)
    return list
    
A = np.random.rand(5, 5)
b = np.random.rand(5)
L, U = my_lu(A)
y = fsubst(L, b)
x = b_subst(U, y)
x3 = np.linalg.solve(A, b)
print(A)
print("----------------------------------------------------------")
print(L)
print("----------------------------------------------------------")
print(U)
print("----------------------------------------------------------")
print(x)
print("----------------------------------------------------------")
print(x3)

[[0.20552485 0.38208745 0.09666294 0.61759545 0.01697467]
 [0.19143192 0.26080681 0.91420104 0.13930783 0.08225586]
 [0.01546235 0.64323327 0.97743673 0.37691735 0.04499574]
 [0.95146422 0.91292499 0.42645479 0.64279275 0.55470963]
 [0.58063737 0.74454274 0.1927075  0.10980207 0.98671518]]
----------------------------------------------------------
[[ 1.          0.          0.          0.          0.        ]
 [ 0.93142958  1.          0.          0.          0.        ]
 [ 0.07523347 -6.46279681  1.          0.          0.        ]
 [ 4.62943648  9.00208176 -1.18163322  1.          0.        ]
 [ 2.82514441  3.52236942 -0.47381208  1.03834318  1.        ]]
----------------------------------------------------------
[[ 0.20552485  0.38208745  0.09666294  0.61759545  0.01697467]
 [ 0.         -0.09508074  0.82416632 -0.43593884  0.06644515]
 [ 0.          0.          6.29658388 -2.48693065  0.47314017]
 [ 0.          0.          0.         -1.23060895  0.43705996]
 [ 0.          0.      

In [2]:
# Doplňte podle bodu 2)
print(A-(L @ U))

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  3.33066907e-16 -1.11022302e-16
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -3.33066907e-16  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.11022302e-16  1.11022302e-16
  -1.11022302e-16]]


In [ ]:
# Doplňte podle bodu 3)
#viz 1)

In [ ]:
# Doplňte podle bodu 4)

#viz 1)

In [44]:
# Doplňte podle bodu 5)
#viz 1)

[ 0.72263408  0.03953983 -0.18897622 -0.54612439  1.51612269]
